# Simulation of PGA Tour Golf Data

## Programming for Data Analysis Project 

### Andrew Walker - G00398788@gmit.ie

![www.pgatour.com](https://pga-tour-res.cloudinary.com/image/upload/c_fill,dpr_2.6,f_auto,g_center,h_336,q_auto,w_1320/v1/pgatour/editorial/2020/12/17/Logo/flag-example-1400.jpg)

This project will simulate a data set based on PGA Tour golf statistics and investigate the variables.

The project will make a number of hypotheses and simulate future data based on historical stats.

https://www.itl.nist.gov/div898/handbook/eda/section3/eda366.htm

https://www.usga.org/content/dam/usga/pdf/RSI/distance-insights-2021-docs/2020%20Distance%20Report%2026-01-2021.pdf

https://www.usga.org/content/dam/usga/pdf/2020/distance-insights/DIPR-FINAL-2020-usga.pdf

https://www.pgatour.com/stats/stat.101.y2021.html

1990 (no major innovations between 1980 and 1990), 2000, 2010, 2020 - simulate data for 2030

Make a note of total number of players on Fedex

In [2]:
# Regular expressions.
import re

# Convenient HTTP requests.
import requests as rq

# Dates and times
import datetime as dt

# Data frames
import pandas as pd

# FOr downloading
import urllib.request as urlrq

#numpy
import numpy as np

# pyplot
import matplotlib.pyplot as plt

# seaborn
import seaborn as sns

import math

#insall fitter 
#https://pypi.org/project/fitter/

from scipy import stats
#https://stackoverflow.com/questions/16330831/most-efficient-way-to-find-mode-in-numpy-array
    
import scipy

rng = np.random.default_rng()

from fitter import get_common_distributions
get_common_distributions()
from fitter import Fitter





### Compile stats fop top 100 Fedex Cup rankings

https://www.pgatour.com/content/pgatour/stats/stat.02671.y2021.html

In [4]:
df = pd.read_csv('data\ranking.csv')
print(df.to_string()) 

OSError: [Errno 22] Invalid argument: 'data\ranking.csv'

In [ ]:
#arrange as data frame
df_arr = df
df_arr.columns = ['Fedex Rank 2021', 'Name', 'Driving distance (avg yds)','Driving accuracy (%)','Greens In Regulation (%)', 'Birdie average']
#https://note.nkmk.me/en/python-pandas-t-transpose/
df_arr

In [ ]:
distance = df_arr.iloc[:,2]


In [ ]:
accuracy = df_arr.iloc[:,3]


In [ ]:
accuracy_mode = stats.mode(accuracy)
#https://stackoverflow.com/questions/16330831/most-efficient-way-to-find-mode-in-numpy-array


In [ ]:
accuracy_mean = accuracy.mean()
print(accuracy_mean)

In [ ]:
gir = df_arr.iloc[:,4]


In [ ]:
birdies = df_arr.iloc[:,5]


In [ ]:
my_array = np.array([[distance],[accuracy],[gir],[birdies]])               
my_array

In [ ]:
df = pd.DataFrame(np.concatenate(my_array))
print(df)
print(type(df))

In [ ]:
df_t = df.T
df_t.columns = ['Driving distance (avg yds)','Driving accuracy (%)','Greens In Regulation (%)', 'Birdie average']
#https://note.nkmk.me/en/python-pandas-t-transpose/

In [ ]:
df_t.head()

In [ ]:
sns.pairplot(df_t, kind="reg", plot_kws={'line_kws':{'color':'red'}})

In [ ]:
#from scipy import stats
#data = stats.gamma.rvs(2, loc=1.5, scale=2, size=10000)
#print(data)

In [ ]:
fitdistance = np.array([distance])


In [ ]:
#https://medium.com/the-researchers-guide/finding-the-best-distribution-that-fits-your-data-using-pythons-fitter-library-319a5a0972e9
#https://pypi.org/project/fitter/
#https://fitter.readthedocs.io/en/latest/faqs.html#what-are-the-distributions-available

f = Fitter(fitdistance, distributions=['cauchy',
 'chi2',
 'expon',
 'exponpow',
 'gamma',
 'lognorm',
 'norm',
 'powerlaw',
 'rayleigh',
 'uniform'])
f.fit()
# may take some time since by default, all distributions are tried
# but you call manually provide a smaller set of distributions
f.summary()

In [ ]:
f.get_best(method = 'sumsquare_error')

In [ ]:
fitaccuracy = np.array([accuracy])

In [ ]:
f = Fitter(fitaccuracy, distributions=['cauchy',
 'chi2',
 'expon',
 'exponpow',
 'gamma',
 'lognorm',
 'norm',
 'powerlaw',
 'rayleigh',
 'uniform'])
f.fit()
# may take some time since by default, all distributions are tried
# but you call manually provide a smaller set of distributions
f.summary()

In [ ]:
al = f.get_best(method = 'sumsquare_error')
al


In [ ]:
fitgir = np.array([gir])

In [ ]:
f = Fitter(fitgir, distributions=['cauchy',
 'chi2',
 'expon',
 'exponpow',
 'gamma',
 'lognorm',
 'norm',
 'powerlaw',
 'rayleigh',
 'uniform'])
f.fit()
# may take some time since by default, all distributions are tried
# but you call manually provide a smaller set of distributions
f.summary()

In [ ]:
fitbirdies = np.array([birdies])

In [ ]:
f = Fitter(fitbirdies, distributions=['cauchy',
 'chi2',
 'expon',
 'exponpow',
 'gamma',
 'lognorm',
 'norm',
 'powerlaw',
 'rayleigh',
 'uniform'])
f.fit()
# may take some time since by default, all distributions are tried
# but you call manually provide a smaller set of distributions
f.summary()

In [ ]:
df = pd.read_csv('data\driving_stats.csv')

print(df.to_string()) 

Name
Driving distance avg (https://www.pgatour.com/content/pgatour/stats/stat.101.y2021.html)
Driving accuracy % (https://www.pgatour.com/content/pgatour/stats/stat.102.y2021.html)
GIR (https://www.pgatour.com/content/pgatour/stats/stat.103.y2021.eoff.t013.html)
Fedex Cup Regular Season Standings (https://www.pgatour.com/content/pgatour/stats/stat.02394.y2021.html)

In [ ]:
df_arr = df
df_arr.columns = ['Driving distance avg (yds)','Driving accuracy (%)','Scoring avg (%)','GIR']
#https://note.nkmk.me/en/python-pandas-t-transpose/

In [ ]:
df_arr

In [ ]:
# Write a function / loop to plot these histograms
# Investigate how to determine what their distribution is... Skilearn? 

In [ ]:
#distance2020 = df_arr.iloc[:,0]
#distance2020
x = df_arr.iloc[:,0]
x

In [ ]:
plt.hist(x)

In [ ]:
#accuracy2020 = df_arr.iloc[:,1]
#accuracy2020
y = df_arr.iloc[:,1]
y

In [ ]:
plt.hist(y)

In [ ]:
#plt.scatter(distance2020, accuracy2020)
#m, b = np.polyfit(distance2020, accuracy2020, 1)
#plt.plot(distance2020, m*distance2020 + b)
#plt.show()
#https://www.kite.com/python/answers/how-to-plot-a-linear-regression-line-on-a-scatter-plot-in-python

#plt.scatter(z,y)
#m, b = np.polyfit(z,y, 1)
#plt.plot(z, m*z + b)
#plt.show()
#https://www.kite.com/python/answers/how-to-plot-a-linear-regression-line-on-a-scatter-plot-in-python



In [ ]:
z = df_arr.iloc[:,2]
z

In [ ]:
# make a note that low scoring is better
plt.hist(z)

In [ ]:
plt.scatter(y,z)

plt.show()

In [ ]:
plt.scatter(y,x)
plt.show()

In [ ]:
plt.scatter(y,x)
m, b = np.polyfit(y, x, 1)
plt.plot(y, m*y + b)
plt.show()

In [ ]:
zz = df_arr.iloc[:,3]
zz

In [ ]:
plt.hist(zz)

In [ ]:
plt.scatter(z,zz)

plt.show()

In [ ]:
my_array = np.array([[x],[y],[z],[zz]])
                    
my_array

In [ ]:
df = pd.DataFrame(np.concatenate(my_array))
print(df)
print(type(df))

In [ ]:
df_t = df.T
df_t.columns = ['Driving distance avg (yds)','Driving accuracy (%)','Scoring avg (%)','GIR']
#https://note.nkmk.me/en/python-pandas-t-transpose/

In [ ]:
df_t.head()

In [ ]:
sns.pairplot(df_t, kind="reg", plot_kws={'line_kws':{'color':'red'}})

### Distance

In [ ]:
# Driving distacnce
#https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.lognormal.html#numpy.random.Generator.lognormal
mu, sigma = 316.5, 5.1

distance_rand = rng.lognormal(mu, sigma, 100)

count, bins, ignored = plt.hist(distance_rand, 100, density=True)

x = np.linspace(min(bins), max(bins), 100)

pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))

       / (x * sigma * np.sqrt(2 * np.pi)))

plt.plot(x, pdf, linewidth=2, color='r')

plt.axis('tight')

plt.show()



### Accuracy

In [ ]:
from scipy.stats import rayleigh

In [ ]:
# https://www.kite.com/python/answers/how-to-convert-a-dictionary-to-a-numpy-array-in-python 
# use this to put the scale factor in
al_data = list(al.items())
al_array = np.array(al_data)
al_array

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rayleigh.html
accuracy_rand = rayleigh.rvs(loc=62.6, scale=6.319240523124456, size=100, random_state=None) # use the mean value determined for the loc and the scale is determined using fitter
plt.hist(accuracy_rand, bins=10)

plt.show()

In [ ]:
type(al)

### GIR

In [ ]:
# GIR
mu, sigma = 70.2, 1.5

gir_rand = np.random.default_rng().normal(mu, sigma, 100)

count, bins, ignored = plt.hist(gir_rand, 10, density=True)

plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *

               np.exp( - (bins - mu)**2 / (2 * sigma**2) ),

         linewidth=2, color='r')

plt.show()

# https://www.statisticshowto.com/choose-bin-sizes-statistics/

In [ ]:
accuracy_mode

In [ ]:
plt.hist(accuracy)

In [ ]:
plt.scatter(distance_rand, accuracy_rand)
m, b = np.polyfit(distance_rand, accuracy_rand, 1)
plt.plot(distance_rand, m*distance_rand + b)
plt.show()
#https://www.kite.com/python/answers/how-to-plot-a-linear-regression-line-on-a-scatter-plot-in-python

In [ ]:
y

In [ ]:
y.max(), y.min(), len(y)


In [ ]:
plt.scatter(x,y)
m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b)
plt.show()
#https://www.kite.com/python/answers/how-to-plot-a-linear-regression-line-on-a-scatter-plot-in-python

In [ ]:
mu, sigma = 69.4, 0.4

z = np.random.default_rng().normal(mu, sigma, 1000)

count, bins, ignored = plt.hist(z, 10, density=True)

plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *

               np.exp( - (bins - mu)**2 / (2 * sigma**2) ),

         linewidth=2, color='r')

plt.show()

In [ ]:
mu, sigma = 70.2, 1.5

zz = np.random.default_rng().normal(mu, sigma, 1000)

count, bins, ignored = plt.hist(zz, 10, density=True)

plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *

               np.exp( - (bins - mu)**2 / (2 * sigma**2) ),

         linewidth=2, color='r')

plt.show()

In [ ]:
my_array = np.array([[x],[y],[z],[zz]])
                    
my_array

In [ ]:
#irisData = pd.read_csv('bezdekIris.data', header = None)  
#irisData.columns = ['Sepal Length (cm)','Sepal Width (cm)','Petal Length (cm)','Petal Width (cm)','Class']

In [ ]:
df = pd.DataFrame(np.concatenate(my_array))


print(df)
print(type(df))

In [ ]:
df_t = df.T
df_t.columns = ['Driving distance avg (yds)','Driving accuracy (%)','Scoring avg (%)','GIR']
#https://note.nkmk.me/en/python-pandas-t-transpose/

In [ ]:
df_t.head()

In [ ]:
sns.pairplot(df_t, kind="reg", plot_kws={'line_kws':{'color':'red'}})
# https://stackoverflow.com/questions/50722972/change-the-regression-line-colour-of-seaborns-pairplot